# 👨‍💼👨‍🚀👨‍💻 RAG with File Retrieval

This notebook is restricted to OpenAI API only!

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.15.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.15.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.15.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.15.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.15.1-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;

var kernel = Kernel.CreateBuilder()
    // .AddAzureOpenAIChatCompletion(
    //     deploymentName: Secrets.DeploymentName,
    //     endpoint: Secrets.Endpoint,
    //     apiKey: Secrets.ApyKey)
    .AddOpenAIChatCompletion(
        modelId: Secrets.OpenAIModel,
        apiKey: Secrets.OpenAIApiKey)
    .Build();

### Let's create a chat group for agents and add a goal to the chat.

In [ ]:
#pragma warning disable SKEXP0110

var goal = """
    Here’s a recipe for low carb pancakes with quantities:

    Ingredients:

    2 eggs
    1 cup almond flour
    1 teaspoon vanilla extract
    1 teaspoon baking powder
    1 tbsp. Confectioners Swerve
    Pinch of salt
    3 tablespoon butter, melted
    Instructions:

    In a medium bowl, whisk together the eggs, almond flour, vanilla extract, baking powder, Confectioners Swerve, and salt.
    Melt the butter and add it to the mixture, whisking until well combined.
    Heat a non-stick skillet over medium heat.
    Spoon a portion of the pancake mix onto the hot skillet.
    Cook until bubbles form on the surface, then flip and cook until browned on the other side.
    Repeat with the remaining batter.
    Serve your low carb pancakes with your favorite low-carb syrup or fresh berries.
    Enjoy your meal! 😊
    """;

AgentGroupChat chat = new();

// short-term memory
chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, goal));


### Read _USDA.pdf_ file and upload it to the model

In [ ]:
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0001

// uploading the file content (byte array) to the service
var fileService = new OpenAIFileService(Secrets.OpenAIApiKey);
var bytes = await System.IO.File.ReadAllBytesAsync("../../Resources/Data/USDA.pdf");
var binaryContent = new BinaryContent(bytes, mimeType: "application/pdf");
var executionSettings = new OpenAIFileUploadExecutionSettings("../../Resources/Data/USDA.pdf", OpenAIFilePurpose.Assistants);
var uploadFile = await fileService.UploadContentAsync(
    binaryContent,
    executionSettings
);
// this is the long-term memory

// we will refer the uploaded file by its id

### Create nutritionist agent with file retrieval capabilities (agents are using the out-of-the-box RAG feature of the OpenAI assistants)

In [ ]:
#pragma warning disable SKEXP0110

var agent = await OpenAIAssistantAgent.CreateAsync(
    kernel: kernel,
    config: new OpenAIAssistantConfiguration(Secrets.OpenAIApiKey!),
    definition: new OpenAIAssistantDefinition
    {
        Instructions = """
            You are an expert nutritionist.
            If you are provided with meal recipes, you respond with the number of calories for the provided meal recipe.
            If you are not provided with meal recipes then you politely suggest the user to input a meal recipe.
            """,
        Description = "An expert nutritionist.",
        Name = "Nutritionist",
        Id = "Nutritionist_01",
        EnableRetrieval = true,
        FileIds = [uploadFile.Id],
        ModelId = Secrets.OpenAIModel
    });

### ⚡ The chat is invoking nutitionist agent

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

Console.WriteLine($"{AuthorRole.User} > {goal}.");

chat.AddAgent(agent);

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"\n{content.Role} [{content.AuthorName}] > {content.Content}");
}

### Notice that the agent is fetching additional context from the external file _USDA.pdf_